<a href="https://colab.research.google.com/github/o-fugi/FURSPColexification/blob/main/code/Prediction_with_GMM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/ColabFiles/

Mounted at /content/drive/
/content/drive/MyDrive/ColabFiles


In [2]:
import pandas as pd
import re
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from functools import reduce
import matplotlib as mpl
import torch

In [3]:
# install sentence-trasnformers
%%capture
! pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sentence_transformers import util
model = SentenceTransformer('whaleloops/phrase-bert')

In [8]:
# import semantic shift dataset

sem_shift_df = pd.read_csv('/content/drive/MyDrive/ColabFiles/Project 10 Datasets/cleaned_dat_sem_shift.csv')
sem_shift_df['meaning1'] = sem_shift_df['meaning1_clean']
sem_shift_df['meaning2'] = sem_shift_df['meaning2_clean']

sem_shift_df.at[697, 'meaning1'] = 'furuncul'
sem_shift_df.at[1521, 'meaning2'] = 'geometrid'

shift_class_df = sem_shift_df

## PCA

In [9]:
# create a dictionary for the embeddings
vec_dic = {} # This will be a dictionary that easily allows us to access the embedding for all of our senses, saving time. 
error_senses = set()  # This represents the set of senses for which there was a problem converting them to embeddings or concreteness values

for i in range(len(shift_class_df)): # Here we loop through each row of our dataframe, and if we can convert a sense s to an embedding then we set vec_dic[s] = embedding
  row = shift_class_df.iloc[i]
  x = row["meaning1"]
  y = row["meaning2"]

  try:   
    if x not in vec_dic:
      xvec = np.array(model.encode(x))
      vec_dic[x] = xvec
  except:
    error_senses.add(x)

  try:  
    if y not in vec_dic:
      yvec = np.array(model.encode(y))
      vec_dic[y] = yvec
  except: 
    error_senses.add(y)

error_senses = list(error_senses) # List of all senses that could not be converted to embeddings. Should be empty right now with phrase BERT
senses = list(vec_dic.keys()) # List of all concepts

sense_indices = {senses[i]:i for i in range(len(senses))} # sense_indices is a dictionary where its keys are senses and its values are the indices for which the senses appear in our list of senses.

In [10]:
# create dataframe with a pair of embeddings for each shift

all_vars_df = pd.DataFrame()
all_vars_df['meaning1'] = shift_class_df['meaning1']
all_vars_df['meaning2'] = shift_class_df['meaning2']

# #if working with the English database, these are helpful
# all_vars_df['word'] = shift_class_df['Word']
# all_vars_df['type'] = shift_class_df['Type of change']

vec_df = pd.DataFrame.from_dict(vec_dic, orient='index').reset_index().rename(columns={'index':'Word'})
vec_meaning_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning1',right_on='Word', how='left'), [all_vars_df, vec_df])
vec_meaning_df = reduce(lambda  left,right: pd.merge(left,right,left_on='meaning2',right_on='Word', how='left'), [vec_meaning_df, vec_df])
vec_meaning_df = vec_meaning_df.drop(['Word_x', 'Word_y'], axis=1)

In [11]:
# create dataframe with one difference embedding for each shift

vec_diff_df = pd.DataFrame()

for i in range(len(model.encode('yikes'))):
  vec_diff_df[i] = vec_meaning_df[str(i) + "_y"] - vec_meaning_df[str(i) + "_x"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  


In [12]:
# create dataframe with all shifts and difference vectors 

source_shift_df = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True, how='left'), [all_vars_df, vec_diff_df])

In [13]:
# perform pca, just so the gaussian mixture will perform a little better
# x = source_shift_df.drop(['word', 'meaning1', 'meaning2', 'type'], axis=1).values # if working with English database
x = source_shift_df.drop(['meaning1', 'meaning2'], axis=1).values
scaler = StandardScaler()
scaler.fit(x)
x_scale = scaler.transform(x)

# do PCA
pca = PCA(n_components=50)
pca.fit(x_scale)
components = pca.transform(x_scale)
components_df = pd.DataFrame(data = components)#.rename(columns={0:'PC_1' , 1:'PC_2', 2:"PC_3", 3:'PC_4'})

# merge back into word data
df = reduce(lambda  left,right: pd.merge(left,right, left_index=True, right_index=True), [all_vars_df, components_df])

## GMM 

In [15]:
# group the shifts

n_dimensions = 50
n_components = 17
estimator = GaussianMixture(n_components=n_components, covariance_type='spherical', init_params='kmeans', max_iter=20) # other covariance is "spherical", "diag", "tied"
estimator.fit(df[range(n_dimensions)])

/usr/local/lib/python3.7/dist-packages/sklearn/mixture/_base.py:282: ConvergenceWarning: Initialization 1 did not converge. Try different init parameters, or increase max_iter, tol or check for degenerate data.
  ConvergenceWarning,


GaussianMixture(covariance_type='spherical', max_iter=20, n_components=17)

In [109]:
label_df = pd.DataFrame(estimator.predict(np.asarray(df[range(50)])))
label_df = label_df.rename(columns={0:'label'})

df = reduce(lambda  left,right: pd.merge(left,right,left_index=True,right_index=True, how='left'), [df, label_df])

## brute force and GMM models

In [230]:
# create train and test dfs

train_df = df.sample(frac = 0.8)
test_df = df.drop(train_df.index)

train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

In [239]:
def brute_force(source, potential_targets):
  average_dists = []

  for target in potential_targets:
    diff_vec = vec_dic[target] - vec_dic[source]
    diff_vec_pca = pca.transform(scaler.transform([diff_vec])) # pca transform
    diff_df = pd.DataFrame([diff_vec_pca[0]])
    diff_df = pd.DataFrame(np.repeat(diff_df.values, len(train_df), axis=0), columns=diff_df.columns)

    dists = np.linalg.norm(train_df[list(range(50))] - diff_df, axis=1)

    average_dists.append(np.average(dists))
  
  best_target = np.argmin(average_dists)

  return best_target

def dist_from_closest(source, potential_targets):
  min_dists = []

  for target in potential_targets:
    diff_vec = vec_dic[target] - vec_dic[source]
    diff_vec_pca = pca.transform(scaler.transform([diff_vec])) # pca transform
    diff_df = pd.DataFrame([diff_vec_pca[0]])
    diff_df = pd.DataFrame(np.repeat(diff_df.values, len(train_df), axis=0), columns=diff_df.columns)

    dists = np.linalg.norm(train_df[list(range(50))] - diff_df, axis=1)
    # print(source, "->", target, "is similar to", train_df.iloc[np.argmin(dists)][['meaning1', 'meaning2']].values)
    
    min_dists.append(min(dists))
  
  best_target = np.argmin(min_dists)
  
  # print(min_dists)

  return best_target

def gmm_model(source, potential_targets):
  average_dists = []

  for target in potential_targets:
    diff_vec = vec_dic[target] - vec_dic[source]
    diff_vec_pca = pca.transform(scaler.transform([diff_vec])) # pca transform

    # closest cluster, by GMM automatic labelling 
    cluster = estimator.predict(diff_vec_pca)[0]

    # filter train dataset to only include that cluster 
    train_df_source = train_df[train_df['label']==cluster]

    diff_df = pd.DataFrame([diff_vec_pca[0]])
    diff_df = pd.DataFrame(np.repeat(diff_df.values, len(train_df_source), axis=0), columns=diff_df.columns)

    dists = np.linalg.norm(train_df_source.reset_index()[list(range(50))] - diff_df, axis=1)

    average_dists.append(np.average(dists))
  
  best_target = np.argmin(average_dists)

  return best_target

def gmm_dist_from_center(source, potential_targets):
  dists = []

  for target in potential_targets:
    diff_vec = vec_dic[target] - vec_dic[source]
    diff_vec_pca = pca.transform(scaler.transform([diff_vec])) # pca transform

    # closest cluster, by GMM automatic labelling 
    cluster = estimator.predict(diff_vec_pca)[0]

    dist = np.linalg.norm(estimator.means_[cluster] - diff_vec_pca)

    dists.append(dist)
  
  best_target = np.argmin(dists)

  return best_target

In [240]:
num_correct_brute = 0
num_correct_dist_closest = 0
num_correct_gmm = 0
num_correct_gmm_dist = 0

# use cosine similarity / euclidean distance
# use distance from closest vector / average of distances from all vectors  

for i in range(len(test_df)):
  row = test_df.iloc[i]
  source = row['meaning1']
  
  # make list of potential targets to choose from
  potential_targets = []
  potential_targets.append(row['meaning2']) # the ACTUAL target
  df_targets = df[df['meaning1']!=source] # only take elements of the dataframe that don't have source as meaning1 
  potential_targets += list(df_targets['meaning2'].sample(n=4)) # sample 4 random targets

  average_dists = []

  if brute_force(source, potential_targets)==0:
    num_correct_brute +=1

  if dist_from_closest(source, potential_targets)==0:
    num_correct_dist_closest +=1
  
  if gmm_model(source, potential_targets)==0:
    num_correct_gmm += 1

  if gmm_dist_from_center(source, potential_targets)==0:
    num_correct_gmm_dist +=1

active -> clever is similar to ['active' 'keen of senses']
active -> to prevent is similar to ['empty' 'in vain']
active -> parachute is similar to ['frog' 'toad']
active -> day is similar to ['predator' 'wolf']
active -> week is similar to ['to feel' 'to feel sorry']
[23.835774567368937, 25.862682837303247, 26.260430366276744, 22.22223024925067, 24.244798166515185]
acute of sound -> spicy is similar to ['deaf' 'stupid']
acute of sound -> to do is similar to ['deaf' 'stupid']
acute of sound -> daughter is similar to ['to give' 'to marry off a daughter']
acute of sound -> to deceive is similar to ['to blow' 'to deceive']
acute of sound -> speech is similar to ['to make noise' 'to speak']
[19.59631116727076, 24.3663348971393, 26.38655466165533, 25.757386525513947, 21.964289039595684]


In [243]:
print(num_correct_brute/len(test_df))
print(num_correct_dist_closest/len(test_df))
print(num_correct_gmm/len(test_df))
print(num_correct_gmm_dist/len(test_df))

0.0
0.5
0.0
0.0
